<h1>Apache Spark on GCP</h1>


<h1>The data</h1>
<li>NYC yellow cab trip data</li>
<li>Yellow cab data is available from <a href="https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page">https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page</a></li>
<li>We're only interested in data from 2017</li>
<h1>The problem</h1>
<li>Calculate the mean and the variance of the trip cost for trips originating in each taxi zone</li>
<li>The PULocationID column corresponds to the taxi zone</li>
<li>The trip cost is the fare_amount plus the tip_amount. You can ignore all other charges</li>
<li>You must use combineByKey for this problem. And you must use the one pass method for calculating both the mean as well as the variance. (i.e., you should make only one call to combineByKey)</li>
<li>Use the following formula for calculating variance: $ variance = \frac{\sum x_i^2 -n\bar{x}^2}{(n-1)} $ </li>
<li>The program should output a Map of (zone -> (mean,variance), zone  -> (mean,variance) , ......</li>

Example of output:

Map(188 -> (14.666259597276545,249.33821353365235), 204 -> (55.0072972972973,2666.6166251278305), 194 -> (42.80580259365995,915.6050500618951), ....

<h1>Getting the data</h1>
<li>Use the following shell script to get the data into your storage bucket</li>
<li>Note: Replace my bucket uri with your bucket uri</li>
<b>Make sure that the entire curl command is in a single line!</b> Mine should be:
<br>
<small>`curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_$val.csv | gsutil cp - gs://hj2203-ieor4526-bucket-01/data/yellow_tripdata$val.csv`</small>
<br>
Also note that the single quote at the front and back of the curl statement is a backquote, not the regular single quote

<h3>Using the shell script</h3>
<li>On your cloud Jupyter, open a new Python 3 notebook</li>
<li>Cut and paste the script below into a cell in that notebook</li>
<li>Make sure you've created a folder named "data" in your cloud storage bucket</li>
<li>modify the script to replace my cloud URI with the name of your storage bucket</li>
<li>Run the cell. It will take some time but, assuming your file references are correct, you should see all the taxi data files in the data folder on your cloud storage bucket</li>



<b>NOTE</b>: The script below will run in a python notebook. Not in a spylon-kernel notebook

In [1]:
// # %%bash
// # declare -a arr=("2017-01" "2017-02" "2017-03" "2017-04" "2017-05" "2017-06" "2017-07" "2017-08" "2017-09" "2017-10"
// #  "2017-11" "2017-12" ) ;
// # for val in ${arr[@]};
// # do
// #    `curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_$val.csv | gsutil cp - gs://hj2203-spark/data/yellow_tripdata$val.csv`
// # done;

Intitializing Scala interpreter ...

Spark Web UI available at http://cluster-e27e-m:8088/proxy/application_1647806180297_0002
SparkContext available as 'sc' (version = 3.1.2, master = yarn, app id = application_1647806180297_0002)
SparkSession available as 'spark'


<h3>The script explained</h3>
<pre>
LINE 1: #!/bin/bash
LINE 2: declare -a arr=("2017-01" "2017-02" "2017-03" "2017-04" "2017-05" "2017-06" "2017-07" "2017-08" "2017-09" "2017-10" "2017-11" "2017-12" ) 
LINE 3: for val in ${arr[@]}
LINE 4: do
LINE 5: `curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_$val.csv | gsutil cp - gs://hj2203-ieor452
6-bucket-01/data/yellow_tripdata$val.csv`
LINE 6: done
</pre>



<li><span style="color:blue">LINE 1</span>: There are many different unix shells (csh, bash, dash, zsh, etc.). When writing a shell script, the first line tells the OS which shell to run. All the shells are stored in /bin (take a look at the contents of /bin using <span style="color:red">ls /bin</span>). </li>
<li><span style="color:blue">LINE 2</span>: <span style="color:red">declare</span> declares a new variable. The <span style="color:red">-a</span> option indicates that the variable is an array. Note that the elements are separated by space rather than a comma</li>
<li><span style="color:blue">LINE 3</span>: A for loop. arr[@] indicates that we want to use the variable arr (declared in line 2) as a collection from which we will draw values. The \$ sign is a peculiarity of unix shell. When declaring a variable, only the name of the variable is used (e.g., x=5) but when using a pre-declared variable, the \$ sign must be appended in front of the variable name (e.g., echo \$x). Note that there should be no spaces in x=5)</li>
<li><span style="color:blue">LINE 4</span>: do indicates the start of the for block</li>
<li><span style="color:blue">LINE 5</span>: curl is a linux command for getting a file from the internet. Here we want to get the file https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_\$val.csv after substituting 2017-01, 2017-02, etc. for \$val (we're using the val variable so need to put a \$ in front - hopefully you can see why!). The -o option specifies the name of the output file. As the data streams into my machine, I'm streaming it out to the storage bucket. gsutil is a GCP app for managing stuff on the virtual machine through command line commands. gsutil cp copies data from one location to another. The - indicates that the data being copied is streaming in from another program (curl, in this case). And the final value, gs://.... is the cloud storage file where we're saving the data</li>
<li><span style="color:blue">LINE 6</span>: done indicates the end of the for block</li>

In [2]:
//SOLUTION GOES HERE!!

//This code constructs a list of files that need to be read
val dates = Array("2017-01","2017-02","2017-03","2017-04","2017-05",
                  "2017-06","2017-07","2017-08","2017-09","2017-10",
                  "2017-11","2017-12")
//Make a list of file names to be accessed from your cloud storage bucket
// Use the ++ concatenation operator (if you want to!)
val files = dates.map(d => "gs://bs3358-ieor4526-bucket/data/yellow_tripdata"++d++".csv")


//data_rdd_array is an array of MapPartitionsRDD. Each MapPartitionRDD corresponds to one data file.
//Make sure that the data in data_rdd_array has the first two rows dropped (header and a blank row)
val data_rdd_array = files.map(f => sc.textFile(f).mapPartitionsWithIndex{ (idx,iter) => if (idx==0) iter.drop(2) else iter})

//all_data combines all the MapPartitionsRDDs in data_rdd_array into a single MapPartitionsRDD
//Use union (see next cell) to combine two rdds
//Use reduce to combine all the rdds in data_rdd_array
val all_data = data_rdd_array.reduce(_ union _)

//Split all rows on comma
val split_data = all_data.map(x => x.split(","))

//Create a map rdd. 
//The key should be the pickup point id (column 7).
//The value the sum of the fare (column 10) and the tip (column 13)
val mapped_data = split_data.map(x => (x(7), x(10).toDouble+x(13).toDouble))

//Write the combiner, the merger, and the mergeAndCombiner

val combiner = (x: Double) => (1,x, x*x) 
val merger = (x: (Int, Double, Double),y:Double) => {
    val (c,acc1, acc2) = x
    (c+1, acc1 + y, acc2 + y*y)
}
val mergeAndCombiner = (x1: (Int, Double, Double), x2: (Int, Double, Double)) => {
    val (c1, acc1, acc3) = x1
    val (c2, acc2, acc4) = x2
    (c1+c2,acc1+acc2, acc3+acc4)
}

//Do the combine by key
val combined_data = mapped_data.combineByKey(combiner,merger,mergeAndCombiner)

//Write a function that returns the variance and the average
//getVarAndAvgFunction = > (String, Double, Double)
val getVarAndAvgFunction = (a: (String, (Int, Double, Double))) => {
    val (x,y) = a
    val n = y._1
    val mean = y._2/n
    val variance = (y._3-n*mean*mean)/n
    (x,(mean,variance)) // here I used (String, (Double, Double)) because it seems collectAsMap needs a key-value pair form
}

//Collect the result
val result = combined_data.map(x => getVarAndAvgFunction(x))

val r = result.collectAsMap

r

dates: Array[String] = Array(2017-01, 2017-02, 2017-03, 2017-04, 2017-05, 2017-06, 2017-07, 2017-08, 2017-09, 2017-10, 2017-11, 2017-12)
files: Array[String] = Array(gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-01.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-02.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-03.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-04.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-05.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-06.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-07.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-08.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-09.csv, gs://bs3358-ieor4526-bucket/data/yellow_tripdata2017-10.csv, gs://bs3358-ieor4526-...


In [3]:
r

res2: scala.collection.Map[String,(Double, Double)] = Map(188 -> (14.666259597276543,249.33821353365235), 204 -> (55.0072972972973,2666.6166251278305), 194 -> (42.80580259365996,915.6050500618946), 90 -> (12.356284077224528,136849.62022189386), 99 -> (52.35909090909092,4379.528771900826), 111 -> (16.039274809160304,273.7748387870754), 167 -> (14.472861247947455,515.4062245308298), 57 -> (24.40454356846473,1111.2365189825414), 210 -> (25.33258503401361,1944.8432632495712), 219 -> (58.66534703196346,2077.5550642403), 84 -> (47.905641025641025,4007.6813015121643), 173 -> (13.861785099581997,321.71716794693333), 78 -> (22.57363007778738,2130.0479849210656), 63 -> (21.352359223300965,438.559615307852), 105 -> (20.524878048780486,217.5336249851279), 149 -> (21.416573116691282,633.804998005363...


In [4]:
// //Example of union
// val t1 =sc.parallelize(Array("hello","bye"))
// val t2 = sc.parallelize(Array("good","fellow"))
// val t3 = t1 union t2
// t3.collect

In [5]:
// val t4 = Array(t1,t2,t3)
// t4.reduce(_ union _).collect